### This project uses Python pyspark, pandas and numpy packages as well as linux shell scripts to extract, transform and load data to IBM Cloud.

In [ ]:
import numpy as np
import pandas as pd
import scipy.io as sio
import os

In [ ]:
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/99.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/100.mat

In [ ]:
!mkdir  good_data
!mv *.mat good_data

In [ ]:
def read_folder(folder):
    data = 'dummy'
    skip = False
    for file in os.listdir(folder):
        file_id = file[:-4]
        mat_file_dict = sio.loadmat(folder+file)
        del data
        for key, value in mat_file_dict.items():
            if 'DE_time' in key or 'FE_time' in key:
                a = np.array(mat_file_dict[key])
                try:
                    data
                except NameError:
                    data = a
                else:
                    if (data.shape[0] != a.shape[0]):
                        print('skipping ' + file_id)
                        skip = True
                        continue
                    data = np.hstack((data,a))
        if skip:
            skip=False
            continue
        id = np.repeat(file_id,data.shape[0])
        id.shape = (id.shape[0],1)
        data = np.hstack((id,data))
        if data.shape[1] == 2:
            zeros = np.repeat(float(0),data.shape[0])
            zeros.shape =(data.shape[0],1)
            data = np.hstack((data,zeros))
        try:
            result
        except NameError:
            result = data
        else:
            result = np.vstack((result,data))
    return result

In [ ]:
good_result = read_folder('./good_data')

In [ ]:
pdf = pd.DataFrame(good_result)
pdf.to_csv('good_result.csv', header=False, index=True)

In [ ]:
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/105.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/106.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/107.mat
!wget http://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/108.mat

In [ ]:
# get all faulty data files
!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
#!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/48k-drive-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done
#!for url in `curl -s csegroups.case.edu/bearingdatacenter/pages/12k-fan-end-bearing-fault-data |grep mat |grep http |awk -F'href="' '{print $2}' |awk -F'">' '{print $1}'`; do wget $url; done


In [ ]:
!mkdir faulty_data
!mv *.mat faulty_data/

In [ ]:
faulty_result = read_folder('./faulty_data')

In [ ]:
pdf = pd.DataFrame(faulty_result)
pdf.to_csv('faulty_result.csv', header=False, index=True)

In [ ]:
# set values for "credentias", "bucket_name" and "endpoint" for data to be uploaded to the cloud
credentials = {
}
bucket_name = ""
endpoint = ""

In [ ]:
import base64
from botocore.client import Config
from boto3 import ibm_boto3
import time

# Create client 
client = ibm_boto3.client(
    's3',
    aws_access_key_id = credentials["cos_hmac_keys"]['access_key_id'],
    aws_secret_access_key = credentials["cos_hmac_keys"]["secret_access_key"],
    endpoint_url=endpoint
)

client.upload_file('good_result.csv', bucket_name, 'good_result.csv')
client.upload_file('faulty_result.csv', bucket_name, 'faulty_result.csv')